# <font  color = "#0093AF">Defaults, Constants, and Tools</font>

SLEEPY relies on a few default settings, numerical constants, and includes a toolbox. These are all found in the main SLEEPY module:
```
sl.Defaults     #Python dictionary containing defaults
sl.Constants    #Python dictionary containing constants
sl.Tools        #Pythong sub-module containing tools
```

These are summarized here

## Setup

In [1]:
# SETUP SLEEPY
import os
os.chdir('../../..')

In [2]:
import SLEEPY as sl
import numpy as np
import matplotlib.pyplot as plt

## Defaults
SLEEPY defaults include data types for real and complex data (`rtype`,`ctype`), whether or not to use parallel processing (`parallel`), whether or not to cache propagators for steps in the rotor cycle (`cache`), the number of cores to use for parallel processing (`ncores`), and whether to print information about calculations (`verbose`).

At the moment, we do not recommend changing the default data types. Numpy does not support higher precision data types for all calculations, and lower precision data types do not result in significant speed up in calculations (although should reduce memory usage). Parallel processing is often not beneficial, as the implemented algorithms in numpy are already fairly optimized for parallel calculations, and parallelism introduces communication overhead. Exceptions may occur for larger powder averages and possibly on different systems. The cache stores propagators for individual steps in the rotor period; it may be turned off if memory usage is too hight. ncores set to None will use all cores available for parallel processing, but an integer may also be provided. Verbosity can be suppressed, especially where many simulations should be performed.

Note that adjusting cache and parallel settings after building a Liouville matrix may result in a propagator cache that is not properly set up.

In [6]:
sl.Defaults

{'rtype': numpy.float64,
 'ctype': numpy.complex128,
 'parallel': False,
 'cache': True,
 'ncores': None,
 'verbose': True}

# Constants

In [7]:
sl.Constants

{'h': 6.62607015e-34,
 'kB': 1.380649e-23,
 'mub': -13996244936.072704,
 'ge': 2.0023193043609235,
 'muen': 5.05078369931e-27}